In [0]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [0]:
import pandas as pd
import numpy as np
import sys
import re

In [0]:
def decode(datatrain):
    decoded_train = []
    for col in datatrain:
        rows = ''
        for row in range(len(col)):
            if row == 0:
                if col[row] == '1':
                    rows = rows + '100'
                elif col[row] == '2':
                    rows = rows + '010'
                elif col[row] == '3':
                    rows = rows + '001'
            elif row == 1:
                if col[row] == '1':
                    rows = rows + '1000'
                elif col[row] == '2':
                    rows = rows + '0100'
                elif col[row] == '3':
                    rows = rows + '0010'
                elif col[row] == '4':
                    rows = rows + '0001'           
            elif row == 2:
                if col[row] == '1':
                    rows = rows + '1000'
                elif col[row] == '2':
                    rows = rows + '0100'
                elif col[row] == '3':
                    rows = rows + '0010'
                elif col[row] == '4':
                    rows = rows + '0001'
            elif row == 3:
                if col[row] == '1':
                    rows = rows + '100'
                elif col[row] == '2':
                    rows = rows + '010'
                elif col[row] == '3':
                    rows = rows + '001'
            elif row == 4:        
                if col[row] == '1':
                    rows = rows + '10'
                elif col[row] == '2':
                    rows = rows + '01'
        decoded_train.append(rows)
    return decoded_train

In [0]:
def create_population(num_Chromosome = 50, bin_length = 15, max_rule = 10):
    population = []
    for i in range(num_Chromosome):
        rule_length = np.random.randint(low = 1, high = max_rule)
        population.append(''.join(str(x) for x in np.random.choice([0, 1], size=(rule_length*15))))
    return population

In [0]:
def calculate_fitness(datatrain, chromosome):
    jumlah = 0
    for i in datatrain:
        suhu = i[0:3]
        waktu = i[3:7]
        langit = i[7:11]
        kelembapan = i[11:14]
        
        rule_length = 15
        for j in range(int(len(chromosome)/rule_length)):
            rule = chromosome[j*rule_length:(j+1)*rule_length]
            suhu1 = rule[0:3]
            waktu1 = rule[3:7]
            langit1 = rule[7:11]
            kelembapan1 = rule[11:14]
           
            trueS = False
            for k in range(len(suhu)):
              if suhu[k] == suhu1[k] and suhu1[k] == '1':
                trueS = True
                break
            trueW = False
            for u in range(len(waktu)):
              if waktu[u] == waktu1[u] and waktu1[u] == '1':
                trueW = True
                break
            trueL = False
            for w in range(len(langit)):
              if langit[w] == langit1[w] and langit1[w] == '1':
                trueL = True
                break
            trueK = False
            for y in range(len(kelembapan)):
              if kelembapan[y] == kelembapan1[y] and kelembapan1[y] == '1':
                trueK = True
                break
            trueT = i[len(i)-1] == rule[len(rule)-1]
            if trueS and trueW and trueL and trueK and trueT:
              jumlah = jumlah + 1
              break
    return jumlah/len(datatrain)
            
            
            
            
            

In [0]:
def crossover(parent1, parent2):
    p1low = np.random.randint(low = 0, high = len(parent1))
    p1high = np.random.randint(low = p1low, high = len(parent1))
    
    
    p1 = p1high - p1low
    gap = p1 % 15
    
    p2prob = [[p1low, p1low+p1], [p1low, p1low+gap], [p1high-p1,p1high], [p1high-gap,p1high]]
    
    p2 = p2prob[np.random.randint(low = 0, high = 4)]
    
    parent1new = parent1[0:p1low] + parent2[p2[0]:p2[1]] + parent1[p1high:len(parent1)]
    parent2new = parent2[0:p2[0]] + parent1[p1low:p1high] + parent2[p2[1]:len(parent2)]
    
    return parent1new, parent2new

In [0]:
def mutasi(kromosom):
    r = random.random()
    if r > 0.01:
      binerrandom = (''.join(str(x) for x in np.random.choice([0,1], size=len(kromosom))))
      biner = ''
      for i in range(len(binerrandom)):
        if binerrandom[i] == '1':
          if kromosom[i] == '0':
            biner = biner + '1'
          elif kromosom[i] == '1':
            biner = biner + '0'
        elif binerrandom[i] == '0':
            biner = biner + kromosom[i]
      return biner

In [0]:
def get_probability(fitness_list):
    total_fitness = np.sum(fitness_list)
    relative_fitness = [f/total_fitness for f in fitness_list]
    
    prob_list = [np.sum(relative_fitness[:i+1]) for i in range (len(relative_fitness))]
    
    return prob_list

In [0]:
def roulette_wheel(population,prob_list,nparent):
    chosen = []
    for _ in range(nparent):
        r = random.random()
        for i in range(len(population)):
            if r <= prob_list[i]:
                chosen.append(population[i])
                break
    return chosen            

In [0]:
def good(population,fitness,datatrain):
    selectedf = []
    p1, p2 = np.sort(fitness)[len(fitness)-2:]
    for i in range(len(population)):
        if calculate_fitness(datatrain,population[i]) == p2:
            selectedf.append(population[i])
            break
    for i in range(len(population)):
        if calculate_fitness(datatrain,population[i]) == p1:
            selectedf.append(population[i])
            break
    return selectedf  

In [0]:
# read data train(data latih) using pandas for easy use
datatrain = pd.read_csv('data_train.csv', header = None)

In [0]:
# convert data train to numpy array, easy to access the value
datatrain = np.asarray(datatrain)

#remove \t (means tab)
regex = re.compile(r'[\n\r\t]')
datatrain = [regex.sub("", i[0]) for i in datatrain]

In [0]:
decoded_datatrain = decode(datatrain)

In [0]:
gen = 100

In [453]:
for i in range(gen):
    fitness = [calculate_fitness(decoded_datatrain, i) for i in population]
    prob = get_probability(fitness)
    new_pop = good(population,fitness, decoded_datatrain)
    while (len(new_pop) < len(population)):
        parent = roulette_wheel(population,prob,2)
        child1, child2 = crossover(parent[0], parent[1])
        offspring1 = mutasi(child1)
        offspring2 = mutasi(child2)
        
        new_pop.append(child1)
        new_pop.append(child2)
    population = new_pop
    
    best = good(population,fitness, decoded_datatrain)
    print("Akurasi", calculate_fitness(decoded_datatrain, best[0])*100, "%")

accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0 %
accuracy 100.0

In [0]:
# read data train(data latih) using pandas for easy use
datauji = pd.read_csv('data_uji.csv', header = None)

In [0]:
# convert data train to numpy array, easy to access the value
datauji= np.asarray(datauji)

#remove \t (means tab)
regex = re.compile(r'[\n\r\t]')
datauji = [regex.sub("", i[0]) for i in datauji]

In [0]:
decoded_datauji = decode(datauji)

In [0]:
def predict(dataset, chromosome):
    predict = []
    for i in dataset:
        suhu = i[0:3]
        waktu = i[3:7]
        langit = i[7:11]
        kelembapan = i[11:14]
        
        rule_length = 15
        for j in range(int(len(chromosome)/rule_length)):
            rule = chromosome[j*rule_length:(j+1)*rule_length]
            
            suhuc = rule[0:3]
            waktuc = rule[3:7]
            langitc = rule[7:11]
            kelembapanc = rule[11:14]
            
            check = rulecheck(list(suhu), list(suhuc)) and rulecheck(list(waktu), list(waktuc)) and rulecheck(list(langit), list(langitc)) and rulecheck(list(kelembapan), list(kelembapanc))
#             print(i, rule)
#             print(rulecheck(list(suhu), list(suhuc)), rulecheck(list(waktu), list(waktuc)), rulecheck(list(langit), list(langitc)), rulecheck(list(kelembapan), list(kelembapanc)))
            if check :
                predict.append(rule[14])
                break
    return predict

In [0]:
pred = predict(decoded_datatrain, best)

In [0]:
def ganti(pred):
  for i in range(len(pred)):
    if pred[i] == '0':
      pred[i] = 'Ya'
    elif pred[i] == '1':
      pred[i] = 'Tidak' 
  return pred 
    

In [0]:
output = ganti(pred)

In [0]:
with open("data_ujihasilpredict.csv", "w") as buka:
  for i in output:
    buka.write(i)